In [3]:
import pandas as pd

df = pd.read_csv("new_netflix_titles.csv")

In [55]:
%load_ext autoreload
%autoreload 2
import selenium
import netflix_collection as helper
import creds
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import creds
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import time

success_dict ={}

def initialize_driver():
    options = webdriver.ChromeOptions()
    proto = "TCP"
    #options = webdriver.ChromeOptions()
    #options.add_argument("--headless")
    #print(os.path.exists())
    options.add_argument("--disable-logging")
    options.add_argument("--window-size=2000,3555") # Needs to be big enough to get all the resolutions
    if (proto == "TCP"):
        options.add_argument("--disable-quic")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")

    #stopthreads =False
    #th = threading.Thread(target=capture_live_packets, args=(url,intface,lambda: stopthreads,))


    driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
    return driver

def check_if_url_valid(driver,page_url):
    start_time= time.time()
    retries = 0
    while(True):
        video_player_length = driver.execute_script(
        r"""return document.getElementsByTagName("video").length""")

        if (video_player_length>0):
            return True
        
        try:
            error_line = driver.find_element("xpath", "/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/span").text
        except:
            error_line = "Unsure"
        if ("Error Code" in error_line):
            return False
        #print(error_line)
        time.sleep(0.5)

        current_time = time.time()
        if (current_time - start_time > 5):
            driver.refresh()
            retries = retries +1
            start_time = current_time





driver = initialize_driver()
collection_start = time.time()
helper.log_in(driver)
url = "https://www.netflix.com/watch"
df = pd.read_csv("new_netflix_titles.csv")
for idx,row in df.iterrows():
    if (row["url_valid"] == True or row["url_valid"] == False):
        print(f"skipping_{row['title']}")
        continue


    if (time.time() - collection_start > 600):
        driver.quit()
        driver = initialize_driver()
        helper.log_in(driver)
        collection_start = time.time()
    
    formatted_url = f"{url}/{row['show_id']}"
    page_url = driver.get(formatted_url)
    is_valid = check_if_url_valid(driver,formatted_url)
    df.at[idx, 'url_valid'] = is_valid
    df.to_csv("new_netflix_titles.csv")
    time.sleep(3)


        


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_1824934/2029547676.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
/tmp/ipykernel_1824934/2029547676.py:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")


LOGIN SUCCESSFUL!
skipping_Norm of the North: King Sized Adventure
skipping_Jandino: Whatever it Takes
skipping_Transformers Prime
skipping_Transformers: Robots in Disguise
skipping_#realityhigh
skipping_Apaches
skipping_Automata
skipping_Fabrizio Copano: Solo pienso en mi
skipping_Fire Chasers
skipping_Good People
skipping_Joaquín Reyes: Una y no más
skipping_Kidnapping Mr. Heineken
skipping_Krish Trish and Baltiboy
skipping_Krish Trish and Baltiboy: Battle of Wits
skipping_Krish Trish and Baltiboy: Best Friends Forever
skipping_Krish Trish and Baltiboy: Comics of India
skipping_Krish Trish and Baltiboy: Oversmartness Never Pays
skipping_Krish Trish and Baltiboy: Part II
skipping_Krish Trish and Baltiboy: The Greatest Trick
skipping_Love
skipping_Manhattan Romance
skipping_Moonwalkers
skipping_Rolling Papers
skipping_Stonehearst Asylum
skipping_The Runner
skipping_6 Years
skipping_Castle of Stars
skipping_City of Joy
skipping_First and Last
skipping_Laddaland
skipping_Next Gen
skippin

/tmp/ipykernel_1824934/2029547676.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
/tmp/ipykernel_1824934/2029547676.py:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")


LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!
LOGIN SUCCESSFUL!


In [46]:

new_netflix_titiles = pd.read_csv("new_netflix_titles.csv")
columns_to_drop = [col for col in new_netflix_titiles.columns if 'Unnamed' in col]
new_netflix_titiles = new_netflix_titiles.drop(columns=columns_to_drop)

In [47]:
new_netflix_titiles["url_valid"].value_counts(normalize=True)

False    0.630574
True     0.369426
Name: url_valid, dtype: float64

In [48]:
working_urls = new_netflix_titiles.query("url_valid == True")
working_urls.value_counts("type")

type
Movie      1325
TV Show     978
dtype: int64

In [50]:
working_urls["genre"] = working_urls['listed_in'].str.split(',').str[0]

/tmp/ipykernel_1979659/2297886059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_urls["genre"] = working_urls['listed_in'].str.split(',').str[0]


In [52]:
top_16_categories = working_urls["genre"].value_counts().head(16).index
working_urls = working_urls[working_urls['genre'].isin(top_16_categories)]

def sample_group(group):
    return group.sample(13) 

sampled_df = working_urls.groupby('genre', group_keys=False).apply(sample_group)
sampled_df = sampled_df.reset_index(drop=True)

In [54]:
sampled_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,url_valid,time_in_seconds,genre
0,80209866,Movie,Manhunt,John Woo,"Zhang Hanyu, Masaharu Fukuyama, Qi Wei, Ji-won...","China, Hong Kong","May 4, 2018",2017,TV-MA,109 min,"Action & Adventure, International Movies",A Chinese pharmaceutical lawyer framed for a v...,True,6540,Action & Adventure
1,80194671,Movie,The Climb,Ludovic Bernard,"Ahmed Sylla, Alice Belaïdi, Kevin Razy, Nicola...",France,"December 29, 2017",2017,TV-14,105 min,"Action & Adventure, Comedies, Dramas",A young Senegalese-French man from humble root...,True,6300,Action & Adventure
2,81033473,Movie,EVANGELION: DEATH (TRUE)²,Hideaki Anno,"Megumi Ogata, Kotono Mitsuishi, Megumi Hayashi...",Japan,"June 21, 2019",1998,TV-14,69 min,"Action & Adventure, Anime Features, Internatio...","Fifteen years after the Second Impact, apathet...",True,4140,Action & Adventure
3,81093113,Movie,NiNoKuni,Yoshiyuki Momose,"Kento Yamazaki, Mackenyu, Mei Nagano, Mamoru M...",Japan,"January 16, 2020",2019,TV-14,107 min,"Action & Adventure, Anime Features, Children &...",Two average teens go on a magical quest to sav...,True,6420,Action & Adventure
4,80149259,Movie,GANTZ:O,"Keiichi Sato, Yasushi Kawamura","Daisuke Ono, M・A・O, Tomohiro Kaku, Saori Hayam...",Japan,"February 18, 2017",2016,TV-MA,96 min,"Action & Adventure, Anime Features, Horror Movies",Teams of recently deceased people who've been ...,True,5760,Action & Adventure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,80117540,TV Show,Atypical,NaN,"Jennifer Jason Leigh, Keir Gilchrist, Michael ...",United States,"November 1, 2019",2019,TV-MA,3 Seasons,"TV Comedies, TV Dramas, Teen TV Shows",When a teen on the autism spectrum decides to ...,True,180,TV Comedies
204,80074503,TV Show,Real Rob,NaN,"Rob Schneider, Patricia Schneider, Jamie Lisso...",United States,"September 29, 2017",2017,TV-MA,2 Seasons,TV Comedies,Go inside the world of comedian and actor Rob ...,True,120,TV Comedies
205,80095532,TV Show,One Day at a Time,NaN,"Justina Machado, Rita Moreno, Todd Grinnell, I...",United States,"February 8, 2019",2019,TV-PG,3 Seasons,TV Comedies,"In a reimagining of the TV classic, a newly si...",True,180,TV Comedies
206,80192577,TV Show,Champions,NaN,"Anders Holm, Fortune Feimster, Andy Favreau, J...",United States,"March 8, 2019",2018,TV-14,1 Season,TV Comedies,"Years after getting his girlfriend pregnant, w...",True,60,TV Comedies


In [53]:
working_urls["genre"].value_counts()

Dramas                      305
Comedies                    270
International TV Shows      240
Stand-Up Comedy             219
Kids' TV                    168
Documentaries               164
Crime TV Shows              164
Action & Adventure          149
Children & Family Movies     90
Docuseries                   85
British TV Shows             72
TV Comedies                  63
Anime Series                 61
Horror Movies                50
Reality TV                   33
International Movies         29
Name: genre, dtype: int64

In [ ]:
working_urls["genre"].value_counts()

Dramas                      305
Comedies                    270
Stand-Up Comedy             219
Documentaries               164
Action & Adventure          149
Children & Family Movies     90
Horror Movies                50
International Movies         29
Movies                       26
Music & Musicals              9
Classic Movies                4
Independent Movies            4
Anime Features                2
Thrillers                     2
Sci-Fi & Fantasy              2
Name: genre, dtype: int64

In [59]:
new_sampled_df = sampled_df[["show_id","genre"]]
new_sampled_df["session_count"] = 0
new_sampled_df.columns = ["video_id","category","session_count"]
new_sampled_df.to_csv("working_urls_netflix.csv")

/tmp/ipykernel_1979659/787389955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sampled_df["session_count"] = 0


In [58]:
new_sampled_df["category"].value_counts()

Action & Adventure          13
Anime Series                13
British TV Shows            13
Children & Family Movies    13
Comedies                    13
Crime TV Shows              13
Documentaries               13
Docuseries                  13
Dramas                      13
Horror Movies               13
International Movies        13
International TV Shows      13
Kids' TV                    13
Reality TV                  13
Stand-Up Comedy             13
TV Comedies                 13
Name: category, dtype: int64